In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from matplotlib import pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm
import warnings

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import metrics


In [2]:

df_path = fr"C:\Users\halik\OneDrive\Dokumenty\GitHub\predikce-nabidky\funcni data set - Copy.xlsx"#@ cesta k datasetu

In [3]:

#@ args:
    # remove_first: odstrani prvních remove first řádků z datasetu (protože nejsou kompletni...)
    # remove_last: odstrani posledni remove_last řádků proto z data setu 
        # default = 6 (momentalne je poslednich 1 radku na predikce)
def get_df(remove_first = 0, remove_last = 1):
    df = pd.read_excel(df_path)
    df = df.drop(df.index[:remove_first])
    if remove_last != 0:
        df = df.drop(df.index[-remove_last:])
    #df['datetime'] = pd.date_range(start='2016-02-01', periods=len(df.index.values), freq='MS')
    return df
df = get_df(remove_first=12)

Vyber modelu

In [7]:

#@ tady mam docela vporadku vypadajici arima summaries 
pdqs = [[1,0,2],[2,5,2]]
for pdq in pdqs:
    p = pdq[0]
    d = pdq[1]
    q = pdq[2]
    model = ARIMA(df['aktivni'], order=(p,d,q), exog=df['predictor']).fit()
    print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                aktivni   No. Observations:                   86
Model:                 ARIMA(1, 0, 2)   Log Likelihood                -657.014
Date:                Sat, 21 Oct 2023   AIC                           1326.027
Time:                        19:00:40   BIC                           1340.753
Sample:                             0   HQIC                          1331.954
                                 - 86                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4693.4534   1420.410      3.304      0.001    1909.502    7477.405
predictor      0.2954      0.135      2.184      0.029       0.030       0.560
ar.L1          0.9721      0.043     22.367      0.0

C:\Users\halik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


train test test

In [26]:
# tady predpovidam na zaklade poslednich mesicu ten nasldeujici(asi nejsem si uplne jistej ze je tohle napsany dobre)
df_train = df.drop(df.index[85:])
df_test = df.drop(df.index[:85])

#@ ARIMA PARAMETRY
p = 1
d = 0
q = 2
model = ARIMA(df_train['aktivni'], order=(p,d,q), exog=df_train['predictor'], missing="raise").fit()
forecast = model.get_forecast(steps=1, exog=df_test["predictor"])



conf_int = forecast.conf_int(alpha=1-(95/100))
conf_int_lower = conf_int['lower aktivni']
conf_int_upper = conf_int['upper aktivni']
forecast_mean = forecast.predicted_mean
print(forecast_mean, df_test["datum"])

97    8046.161518
dtype: float64 97    202303
Name: datum, dtype: int64


In [6]:
ps = [2,3,4]
ds = [0,1,2,3,5]
qs = [1,2,3,4,5]
for p in tqdm(ps):
    for d in ds:
        for q in qs:
            model = ARIMA(df['aktivni'], order=(p,d,q), exog=df['predictor']).fit()
            print(model.summary())

  0%|          | 0/3 [00:00<?, ?it/s]

                               SARIMAX Results                                
Dep. Variable:                aktivni   No. Observations:                   86
Model:                 ARIMA(2, 0, 1)   Log Likelihood                -657.555
Date:                Sat, 21 Oct 2023   AIC                           1327.110
Time:                        18:59:25   BIC                           1341.836
Sample:                             0   HQIC                          1333.036
                                 - 86                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4693.0351   1650.283      2.844      0.004    1458.540    7927.530
predictor      0.2614      0.143      1.831      0.067      -0.018       0.541
ar.L1          0.4985      0.336      1.482      0.1

C:\Users\halik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\halik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:                aktivni   No. Observations:                   86
Model:                 ARIMA(2, 0, 3)   Log Likelihood                -656.810
Date:                Sat, 21 Oct 2023   AIC                           1329.621
Time:                        18:59:26   BIC                           1349.256
Sample:                             0   HQIC                          1337.523
                                 - 86                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4693.5355   1788.775      2.624      0.009    1187.602    8199.469
predictor      0.2389      0.251      0.952      0.341      -0.253       0.730
ar.L1          1.6186      0.804      2.014      0.0

C:\Users\halik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

                               SARIMAX Results                                
Dep. Variable:                aktivni   No. Observations:                   86
Model:                 ARIMA(2, 3, 4)   Log Likelihood                -647.304
Date:                Sat, 21 Oct 2023   AIC                           1310.607
Time:                        18:59:29   BIC                           1329.958
Sample:                             0   HQIC                          1318.381
                                 - 86                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
predictor      0.0934      0.294      0.318      0.751      -0.483       0.670
ar.L1          0.5517      0.215      2.564      0.010       0.130       0.973
ar.L2         -0.7312      0.301     -2.426      0.0

C:\Users\halik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\halik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:                aktivni   No. Observations:                   86
Model:                 ARIMA(2, 3, 5)   Log Likelihood                -646.863
Date:                Sat, 21 Oct 2023   AIC                           1311.727
Time:                        18:59:29   BIC                           1333.496
Sample:                             0   HQIC                          1320.473
                                 - 86                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
predictor      0.1643      0.216      0.759      0.448      -0.260       0.589
ar.L1         -1.6166      0.375     -4.315      0.000      -2.351      -0.882
ar.L2         -0.8932      0.339     -2.634      0.0

C:\Users\halik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
  0%|          | 0/3 [00:04<?, ?it/s]


LinAlgError: LU decomposition error.